In [1]:
import pandas as pd
import traceback
import OpenDartReader
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
dart = OpenDartReader(api_key)

In [2]:
def str2num(x):
    if "," in x:
        x = x.replace(",","")
    if x.startswith("(") and x.endswith(")"):
        x = x.lstrip("(")
        x = x.rstrip(")")
        float_num = float(x) * (-1)
    else:
        x = x.lstrip("(")
        x = x.rstrip(")")     
        float_num = float(x)
    return float_num

In [3]:
reports = dart.list(start='2021-04-01', end='2021-04-30', kind='A', final=True)

In [4]:
reports

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00340096,미래에셋벤처투자,100790,K,분기보고서 (2021.03),20210430001418,미래에셋벤처투자,20210430,
1,01107665,크리스탈신소재,900250,K,사업보고서 (2020.12),20210430001165,크리스탈신소재,20210430,연
2,01022902,애드바이오텍,179530,K,사업보고서 (2020.12),20210430001156,애드바이오텍,20210430,연
3,00799070,이스트아시아홀딩스,900110,K,사업보고서 (2020.12),20210430001050,이스트아시아홀딩스,20210430,연
4,00340005,마이다스아이티,,E,[기재정정]사업보고서 (2020.12),20210430000912,마이다스아이티,20210430,연
...,...,...,...,...,...,...,...,...,...
195,00689418,지유온,111820,E,[기재정정]분기보고서 (2020.09),20210401000667,지유온,20210401,
196,00917861,슈피겐코리아,192440,K,[기재정정]사업보고서 (2020.12),20210401000621,슈피겐코리아,20210401,연
197,00319041,엠허브,,E,소액공모법인결산서류 (2020.12),20210401000449,엠허브,20210401,
198,01183407,이십일스토어,270020,E,사업보고서 (2020.12),20210401000216,이십일스토어,20210401,연


In [5]:
str_expr = "report_nm.str.contains('사업보고서')"
biz_repports = reports.query(str_expr)

In [6]:
biz_repports.shape

(164, 9)

In [7]:
subreports = []
str_expr = "title.str.contains('5. 재무제표 주석')"
for i, row in biz_repports.iterrows():
    try:
        rpt_name = row['report_nm']
        rpt_num = row['rcept_no']        
        rpt_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rpt_num
        df_rpt = dart.sub_docs(rpt_num)
        rpt_data = df_rpt.query(str_expr)       # data frame
        if rpt_data.size == 0:
            subreports.append({'rpt_num' : rpt_num, 'corp_name' : row['corp_name'], 'stock_code' : row['stock_code'], 'rpt_name' : rpt_name, 'rpt_url' : rpt_url,
                            'rpt_title' : None, 'rpt_notes_url' : None})                 
        else:
            rpt_title = rpt_data.iloc[0]['title']
            rpt_notes_url = rpt_data.iloc[0]['url']
            subreports.append({'rpt_num' : rpt_num, 'corp_name' : row['corp_name'], 'stock_code' : row['stock_code'], 'rpt_name' : rpt_name, 'rpt_url' : rpt_url,
                            'rpt_title' : rpt_title, 'rpt_notes_url' : rpt_notes_url}) 
    except Exception as e:
        print(row['rcept_no'])
        print(row['stock_code'])
        print(row['report_nm'])
        print('http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + row['rcept_no'])
        err_msg = traceback.format_exc()
        print(err_msg)

20210409002715
030000
[기재정정]사업보고서 (2020.12)
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20210409002715
Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\urllib3\connection.py", line 454, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\kyungho\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\kyungho\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\kyungho\AppData\Local\Programs\Python\Python310\lib\http\clien

In [8]:
len(subreports)

99

In [9]:
data_list = []
for r in subreports:
    try:
        val_a = val_b = None
        url = r['rpt_notes_url']
        if url == None:
            data_list.append({'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 
                            'val_a' : None, 'val_b' : None})
        else:
            doc_data = pd.read_html(url)
            for df in doc_data:
                row = df.shape[0]
                for i in range(0, row):
                    item = str(df.iloc[i,0])
                    if item.__contains__('확정급여채무의 현재가치'):
                        # print(i, type(df.iloc[i,1]), df.iloc[i,1])
                        str_a = str(df.iloc[i,1])
                        if str_a == "-": val_a = None
                        else: val_a = str2num(str_a)
                    elif item.__contains__('사외적립자산의 공정가치'):
                        # print(i, type(df.iloc[i,1]), df.iloc[i,1])                            
                        str_b = str(df.iloc[i,1])
                        if str_b == "-": val_b = None
                        else: val_b = str2num(str_b)
                    else:
                        continue    
                if (val_a != None) and (val_b != None):
                    break          
            data_list.append({'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 
                            'val_a' : val_a, 'val_b' : val_b})
    except ValueError as err:
        print(r['rpt_num'])
        print(r['rpt_url'])
        print(r['rpt_notes_url'])
        print(err)
        data_list.append({'corp_name' : r['corp_name'], 'stock_code' : r['stock_code'], 'rpt_num' : r['rpt_num'], 'rpt_name' : r['rpt_name'], 
                        'val_a' : val_a, 'val_b' : val_b})        
    except Exception as e:
        print(r['rpt_num'])
        print(r['rpt_url'])
        print(r['rpt_notes_url'])
        err_msg = traceback.format_exc()
        print(err_msg)

20210430001165
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20210430001165
http://dart.fss.or.kr/report/viewer.do?rcpNo=20210430001165&dcmNo=8052458&eleId=17&offset=488140&length=208&dtd=dart3.xsd
Traceback (most recent call last):
  File "C:\Users\kyungho\AppData\Local\Temp\ipykernel_13948\949394539.py", line 10, in <module>
    doc_data = pd.read_html(url)
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\io\html.py", line 1212, in read_html
    return _parse(
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\io\html.py", line 981, in _parse
    tables = p.parse_tables()
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\io\html.py", line 257, in parse_tables
    tables = self._parse_tables(self._build_doc(), self.match, self.attrs)
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\io\html.py", line 816, in _build_doc
    raise e
  File "d:\PythonProject\d

In [10]:
len(data_list)

2

In [45]:
df_all = pd.DataFrame(data_list)

In [46]:
df_all.shape

(48, 6)

In [47]:
df_all

,corp_name,stock_code,rpt_num,rpt_name,val_a,val_b
0,아들과딸,,20210409003049,사업보고서 (2020.12),NaN,NaN
1,제일기획,030000,20210409002715,[기재정정]사업보고서 (2020.12),2.143330e+05,-8.909853e+07
2,휴림네트웍스,192410,20210409002653,[기재정정]사업보고서 (2020.12),NaN,NaN
3,라파스,214260,20210409002282,[기재정정]사업보고서 (2019.12),NaN,NaN
4,탑선,180060,20210409002049,[기재정정]사업보고서 (2020.12),NaN,NaN
5,수성샐바시온,084180,20210409001381,[첨부정정]사업보고서 (2020.12),NaN,NaN
6,KTcs,058850,20210409000489,[기재정정]사업보고서 (2020.12),8.178484e+07,-7.971318e+07
7,웰킵스하이텍,043590,20210409000378,[기재정정]사업보고서 (2019.12),NaN,NaN
8,글로벌에스엠,900070,20210409000239,사업보고서 (2020.12),NaN,NaN
9,현대커머셜,,20210408002358,[기재정정]사업보고서 (2020.12),3.540144e+07,-3.691062e+07


In [11]:
df_all.dtypes

corp_name      object
stock_code     object
rpt_num        object
rpt_name       object
val_a         float64
val_b         float64
dtype: object

In [18]:
# df_all.to_excel('D:/PythonProject/data-gatherer/dart/output/result1.xlsx')
df_all.to_csv('D:/PythonProject/data-gatherer/dart/output/result1.csv')